In [ ]:
import time
import json
from datetime import date

import mechanicalsoup
import stats
import sicris

In [ ]:
date_prefix = date.today().isoformat()
print('Archiving new sicris.json data with timestamp prefix:', date_prefix)

In [ ]:
def get_researchers_ids(oid):
    org_url = 'http://www.sicris.si/public/jqm/search_basic/slv/2/500/search/org/{:s}'.format(oid)
    print(oid, org_url)

    browser = mechanicalsoup.StatefulBrowser()
    browser.open(org_url)
    links = browser.links(link_text='English')
    browser.follow_link(links[0])
    links = browser.links(link_text='Slovene')
    browser.follow_link(links[0])

    links = browser.links(link_text='{JSON}')
    browser.follow_link(links[0])

    page = browser.get_current_page()
    researchers = json.loads(page.text)
    assert len(researchers) == 1
    researchers = researchers[0]
    researchers = researchers['EMPLOY']
    researchers = [r['MSTID'] for r in researchers]
    return researchers

In [ ]:
researchers = get_researchers_ids('1539')

In [ ]:
print('Number of researchers found:', len(researchers))

In [ ]:
def get_researcher_record(rid):
    researcher_url = 'http://www.sicris.si/public/jqm/rsr.aspx?lang=slv&opdescr=search&opt=2&subopt=300&code1=rsr&code2=&psize=10&hits=1&page=1&count=&search_term={:s}&slng=&order_by='.format(rid)
    print(rid, researcher_url)
    browser = mechanicalsoup.StatefulBrowser()
    browser.open(researcher_url)
    links = browser.links('JSON')
    browser.follow_link(links[1])
    page = browser.get_current_page()
    rinfo = json.loads(page.text)
    assert len(rinfo) == 1
    return rinfo[0]

In [ ]:
print('Retriving data on {:d} researchers'.format(len(researchers)))
sicris_data = sicris.get()  # get old data
for rid in researchers:  # update with new info and new researcher entries
    sicris_data[rid] = get_researcher_record(rid)
    sicris_data[rid]['last_update'] = date_prefix
    time.sleep(2)

Store new data into `json` files `data/stats.json` and `data/sicris.json`.

In [ ]:
sicris.update(sicris_data, date_prefix)

In [ ]:
stats.update('researchers', len(researchers), date_prefix)

In [ ]:
print('DONE')